In [1]:
import pandas as pd

products = pd.read_parquet('products.parquet')
return_reason = pd.read_parquet('return_reasons.parquet')
returns = pd.read_parquet('returns.parquet')
reviews = pd.read_parquet('reviews.parquet')
test = pd.read_parquet('test.parquet')

In [2]:
products.info()
products.nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113713 entries, 0 to 113712
Data columns (total 4 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   product_id           113713 non-null  object
 1   category_id          113713 non-null  object
 2   category_title       113713 non-null  object
 3   product_description  96015 non-null   object
dtypes: object(4)
memory usage: 3.5+ MB


product_id             113713
category_id              2930
category_title           4007
product_description     86695
dtype: int64

In [3]:
returns.info()
returns.nunique()

<class 'pandas.core.frame.DataFrame'>
Index: 282141 entries, 0 to 282140
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   id              282141 non-null  int64         
 1   product_id      282141 non-null  object        
 2   cause           282141 non-null  object        
 3   comment         282141 non-null  object        
 4   date_created    282141 non-null  datetime64[ns]
 5   order_item_id   282141 non-null  int64         
 6   customer_id     282141 non-null  object        
 7   purchase_price  282141 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 19.4+ MB


id                282141
product_id         80588
cause                  5
comment            83218
date_created      281921
order_item_id     281958
customer_id       151619
purchase_price      2199
dtype: int64

In [4]:
reviews.info()
reviews.nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502889 entries, 0 to 1502888
Data columns (total 7 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   order_item_id  1502889 non-null  int64         
 1   product_id     1502889 non-null  object        
 2   customer_id    1502889 non-null  object        
 3   review_text    1502888 non-null  object        
 4   shop_id        1502889 non-null  object        
 5   rating         1502889 non-null  int64         
 6   date_created   1502889 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 80.3+ MB


order_item_id    1502889
product_id         79137
customer_id       361393
review_text       704109
shop_id            10037
rating                 5
date_created     1502755
dtype: int64

In [5]:
test.info()
test.nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71323 entries, 0 to 71322
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              71323 non-null  int64 
 1   order_item_id   71323 non-null  int64 
 2   product_id      71323 non-null  object
 3   customer_id     71323 non-null  object
 4   purchase_price  71323 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 2.7+ MB


id                71323
order_item_id     71279
product_id        27455
customer_id       49015
purchase_price     1276
dtype: int64

* Preparing train dataset and test dataset for further processing

In [6]:
rt_kk = returns.drop_duplicates('product_id')[:200]
# mask = reviews['product_id'].isin(rt_kk['product_id'])
# reviews_return_pr = reviews[mask]
mask2 = returns['product_id'].isin(rt_kk['product_id'])
return_df = returns[mask2]
# return_df.nunique()

In [7]:
reviews_return_pr = pd.read_csv('review_return_pr.csv') # we already took translated reviews lets use it

In [8]:
# lets do sentiment analysis
import nltk 
 
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
 
from nltk.corpus import stopwords 
 
from nltk.tokenize import word_tokenize 
 
from nltk.stem import WordNetLemmatizer 
 
import nltk 
 
nltk.download('all') 
 
 
 
def preprocess_text(text): 
 
    tokens = word_tokenize(text.lower()) 
 
    filtered_tokens = [token for token in tokens if token not in stopwords.words('english')] 
 
    lemmatizer = WordNetLemmatizer() 
 
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens] 
 
    processed_text = ' '.join(lemmatized_tokens) 
 
    return processed_text 


analyzer = SentimentIntensityAnalyzer() 
 
def get_sentiment(text): 
 
    scores = analyzer.polarity_scores(text) 
 
    return scores 


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\dustm\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\dustm\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\dustm\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\dustm\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\dustm\AppData\Roaming\nltk_data...
[nltk_data]    | 

In [9]:
def get_sentiment(text): 
 
    scores = analyzer.polarity_scores(text) 
 
    return scores 

In [10]:
reviews_return_pr['preprop'] = reviews_return_pr['translation'].apply(preprocess_text) 
reviews_return_pr['sentiment'] = reviews_return_pr['preprop'].apply(get_sentiment)
reviews_return_pr['sentiment'] = reviews_return_pr['sentiment'].apply(lambda x: x['compound'])

In [11]:
def pos(n):
    if n > 0:
        return n
    else:
        return 0
reviews_return_pr['positive'] = reviews_return_pr['sentiment'].apply(pos)

In [12]:
def neg(n):
    if n < 0:
        return n
    else:
        return 0
reviews_return_pr['negative'] = reviews_return_pr['sentiment'].apply(neg)

In [13]:
def net(n):
    if (n == 0):
        return n
    else:
        return 0
reviews_return_pr['neutral'] = reviews_return_pr['sentiment'].apply(net)

In [14]:
gr_by_product1 = reviews_return_pr.groupby(by = 'product_id').agg({'rating': 'mean', 'sentiment':"mean", 'positive':'mean', 'negative': 'mean', 'neutral': 'mean'}).reset_index(drop=False)
# with_mean = gr_by_product1.merge(reviews_return_pr, on = 'product_id', how = 'right')
# with_mean = with_mean.drop(['rating_y', 'date_created'], axis = 1)
# with_mean.nunique()     # rating_x is mean value of ratings for each product

In [15]:
gr_by_product2 = return_df.groupby(by = 'product_id').agg({'purchase_price': 'mean'})
with_mean_price = gr_by_product2.merge(return_df, on = 'product_id', how = 'right')
with_mean_price = with_mean_price.drop(['purchase_price_y', 'date_created'], axis = 1)
with_mean_price.nunique()

product_id           200
purchase_price_x     181
id                  3259
cause                  5
comment             1730
order_item_id       3251
customer_id         2895
dtype: int64

In [16]:
train_df = with_mean_price.merge(gr_by_product1, on = 'product_id', how = 'left')

In [17]:
train_df.columns

Index(['product_id', 'purchase_price_x', 'id', 'cause', 'comment',
       'order_item_id', 'customer_id', 'rating', 'sentiment', 'positive',
       'negative', 'neutral'],
      dtype='object')

In [18]:
csv_file_path = 'train.csv'  # Desired file name
train_df.to_csv(csv_file_path, index=True)